In [11]:
# Useful starting lines
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load the data

In [34]:
import datetime
from helpers import *
from plots import *
from implementations import *
from preprocessing import *
from preprocessing_naive import *


features, x_train, x_test, y_train, train_ids, test_ids = load_csv_data("data/dataset")

# Split data into train and validation sets
def split_data(x, y, ratio, seed=1):
    """split the dataset based on the split ratio."""
    np.random.seed(seed)
    indices = np.random.permutation(len(y))
    split = int(len(y) * ratio)
    training_idx, test_idx = indices[:split], indices[split:]
    return x[training_idx], y[training_idx], x[test_idx], y[test_idx]

ratio = 0.8
x_train, y_train, x_val, y_val = split_data(x_train, y_train, ratio, seed=1)

In [35]:
print(np.shape(features))
print(np.shape(x_train))
print(np.shape(y_train))
print(np.shape(x_val))
print(np.shape(y_val))
print(np.shape(x_test))

(321,)
(262508, 321)
(262508,)
(65627, 321)
(65627,)
(109379, 321)


In [26]:
# Show the first 5 rows of the data and statistics
pd.set_option('display.max_columns', None)
data = pd.DataFrame(x_train, columns=features)

data.head()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENUM,PVTRESD1,COLGHOUS,STATERES,CELLFON3,LADULT,NUMADULT,NUMMEN,NUMWOMEN,CTELNUM1,CELLFON2,CADULT,PVTRESD2,CCLGHOUS,CSTATE,LANDLINE,HHADULT,GENHLTH,PHYSHLTH,MENTHLTH,POORHLTH,HLTHPLN1,PERSDOC2,MEDCOST,CHECKUP1,BPHIGH4,BPMEDS,BLOODCHO,CHOLCHK,TOLDHI2,CVDSTRK3,ASTHMA3,ASTHNOW,CHCSCNCR,CHCOCNCR,CHCCOPD1,HAVARTH3,ADDEPEV2,CHCKIDNY,DIABETE3,DIABAGE2,SEX,MARITAL,EDUCA,RENTHOM1,NUMHHOL2,NUMPHON2,CPDEMO1,VETERAN3,EMPLOY1,CHILDREN,INCOME2,INTERNET,WEIGHT2,HEIGHT3,PREGNANT,QLACTLM2,USEEQUIP,BLIND,DECIDE,DIFFWALK,DIFFDRES,DIFFALON,SMOKE100,SMOKDAY2,STOPSMK2,LASTSMK2,USENOW3,ALCDAY5,AVEDRNK2,DRNK3GE5,MAXDRNKS,FRUITJU1,FRUIT1,FVBEANS,FVGREEN,FVORANG,VEGETAB1,EXERANY2,EXRACT11,EXEROFT1,EXERHMM1,EXRACT21,EXEROFT2,EXERHMM2,STRENGTH,LMTJOIN3,ARTHDIS2,ARTHSOCL,JOINPAIN,SEATBELT,FLUSHOT6,FLSHTMY2,IMFVPLAC,PNEUVAC3,HIVTST6,HIVTSTD3,WHRTST10,PDIABTST,PREDIAB1,INSULIN,BLDSUGAR,FEETCHK2,DOCTDIAB,CHKHEMO3,FEETCHK,EYEEXAM,DIABEYE,DIABEDU,CAREGIV1,CRGVREL1,CRGVLNG1,CRGVHRS1,CRGVPRB1,CRGVPERS,CRGVHOUS,CRGVMST2,CRGVEXPT,VIDFCLT2,VIREDIF3,VIPRFVS2,VINOCRE2,VIEYEXM2,VIINSUR2,VICTRCT4,VIGLUMA2,VIMACDG2,CIMEMLOS,CDHOUSE,CDASSIST,CDHELP,CDSOCIAL,CDDISCUS,WTCHSALT,LONGWTCH,DRADVISE,ASTHMAGE,ASATTACK,ASERVIST,ASDRVIST,ASRCHKUP,ASACTLIM,ASYMPTOM,ASNOSLEP,ASTHMED3,ASINHALR,HAREHAB1,STREHAB1,CVDASPRN,ASPUNSAF,RLIVPAIN,RDUCHART,RDUCSTRK,ARTTODAY,ARTHWGT,ARTHEXER,ARTHEDU,TETANUS,HPVADVC2,HPVADSHT,SHINGLE2,HADMAM,HOWLONG,HADPAP2,LASTPAP2,HPVTEST,HPLSTTST,HADHYST2,PROFEXAM,LENGEXAM,BLDSTOOL,LSTBLDS3,HADSIGM3,HADSGCO1,LASTSIG3,PCPSAAD2,PCPSADI1,PCPSARE1,PSATEST1,PSATIME,PCPSARS1,PCPSADE1,PCDMDECN,SCNTMNY1,SCNTMEL1,SCNTPAID,SCNTWRK1,SCNTLPAD,SCNTLWK1,SXORIENT,TRNSGNDR,RCSGENDR,RCSRLTN2,CASTHDX2,CASTHNO2,EMTSUPRT,LSATISFY,ADPLEASR,ADDOWN,ADSLEEP,ADENERGY,ADEAT1,ADFAIL,ADTHINK,ADMOVE,MISTMNT,ADANXEV,QSTVER,QSTLANG,MSCODE,_STSTR,_STRWT,_RAWRAKE,_WT2RAKE,_CHISPNC,_CRACE1,_CPRACE,_CLLCPWT,_DUALUSE,_DUALCOR,_LLCPWT,_RFHLTH,_HCVU651,_RFHYPE5,_CHOLCHK,_RFCHOL,_LTASTH1,_CASTHM1,_ASTHMS1,_DRDXAR1,_PRACE1,_MRACE1,_HISPANC,_RACE,_RACEG21,_RACEGR3,_RACE_G1,_AGEG5YR,_AGE65YR,_AGE80,_AGE_G,HTIN4,HTM4,WTKG3,_BMI5,_BMI5CAT,_RFBMI5,_CHLDCNT,_EDUCAG,_INCOMG,_SMOKER3,_RFSMOK3,DRNKANY5,DROCDY3_,_RFBING5,_DRNKWEK,_RFDRHV5,FTJUDA1_,FRUTDA1_,BEANDAY_,GRENDAY_,ORNGDAY_,VEGEDA1_,_MISFRTN,_MISVEGN,_FRTRESP,_VEGRESP,_FRUTSUM,_VEGESUM,_FRTLT1,_VEGLT1,_FRT16,_VEG23,_FRUITEX,_VEGETEX,_TOTINDA,METVL11_,METVL21_,MAXVO2_,FC60_,ACTIN11_,ACTIN21_,PADUR1_,PADUR2_,PAFREQ1_,PAFREQ2_,_MINAC11,_MINAC21,STRFREQ_,PAMISS1_,PAMIN11_,PAMIN21_,PA1MIN_,PAVIG11_,PAVIG21_,PA1VIGM_,_PACAT1,_PAINDX1,_PA150R2,_PA300R2,_PA30021,_PASTRNG,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
0,35.0,10.0,10072015.0,10.0,7.0,2015.0,1100.0,2.015004e+09,2.015004e+09,1.0,1.0,NaN,1.0,2.0,NaN,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,30.0,88.0,30.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,7.0,2.0,NaN,2.0,2.0,2.0,1.0,2.0,2.0,4.0,NaN,2.0,1.0,6.0,1.0,2.0,NaN,1.0,2.0,7.0,88.0,99.0,1.0,180.0,502.0,NaN,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,NaN,NaN,NaN,3.0,888.0,NaN,NaN,NaN,203.0,203.0,203.0,555.0,203.0,201.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,888.0,1.0,2.0,1.0,10.0,1.0,2.0,NaN,NaN,1.0,2.0,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,1.0,5.0,351051.0,17.109993,2.0,34.219987,9.0,NaN,NaN,NaN,1.0,0.726059,76.345588,2.0,9.0,2.0,1.0,2.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,12.0,2.0,78.0,6.0,62.0,1.57,81.65,32.92,4.0,2.0,1.0,4.0,9.0,4.0,1.0,2.0,0.0,1.0,0.0,1.0,0.43,0.43,0.43,0.00,0.43,0.14,0.0,0.0,1.0,1.0,0.86,1.00,2.0,1.0,1.0,1.0,0.0,0.0,2.0,NaN,NaN,19.14,

In [20]:
data.describe()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENUM,PVTRESD1,COLGHOUS,STATERES,CELLFON3,LADULT,NUMADULT,NUMMEN,NUMWOMEN,CTELNUM1,CELLFON2,CADULT,PVTRESD2,CCLGHOUS,CSTATE,LANDLINE,HHADULT,GENHLTH,PHYSHLTH,MENTHLTH,POORHLTH,HLTHPLN1,PERSDOC2,MEDCOST,CHECKUP1,BPHIGH4,BPMEDS,BLOODCHO,CHOLCHK,TOLDHI2,CVDSTRK3,ASTHMA3,ASTHNOW,CHCSCNCR,CHCOCNCR,CHCCOPD1,HAVARTH3,ADDEPEV2,CHCKIDNY,DIABETE3,DIABAGE2,SEX,MARITAL,EDUCA,RENTHOM1,NUMHHOL2,NUMPHON2,CPDEMO1,VETERAN3,EMPLOY1,CHILDREN,INCOME2,INTERNET,WEIGHT2,HEIGHT3,PREGNANT,QLACTLM2,USEEQUIP,BLIND,DECIDE,DIFFWALK,DIFFDRES,DIFFALON,SMOKE100,SMOKDAY2,STOPSMK2,LASTSMK2,USENOW3,ALCDAY5,AVEDRNK2,DRNK3GE5,MAXDRNKS,FRUITJU1,FRUIT1,FVBEANS,FVGREEN,FVORANG,VEGETAB1,EXERANY2,EXRACT11,EXEROFT1,EXERHMM1,EXRACT21,EXEROFT2,EXERHMM2,STRENGTH,LMTJOIN3,ARTHDIS2,ARTHSOCL,JOINPAIN,SEATBELT,FLUSHOT6,FLSHTMY2,IMFVPLAC,PNEUVAC3,HIVTST6,HIVTSTD3,WHRTST10,PDIABTST,PREDIAB1,INSULIN,BLDSUGAR,FEETCHK2,DOCTDIAB,CHKHEMO3,FEETCHK,EYEEXAM,DIABEYE,DIABEDU,CAREGIV1,CRGVREL1,CRGVLNG1,CRGVHRS1,CRGVPRB1,CRGVPERS,CRGVHOUS,CRGVMST2,CRGVEXPT,VIDFCLT2,VIREDIF3,VIPRFVS2,VINOCRE2,VIEYEXM2,VIINSUR2,VICTRCT4,VIGLUMA2,VIMACDG2,CIMEMLOS,CDHOUSE,CDASSIST,CDHELP,CDSOCIAL,CDDISCUS,WTCHSALT,LONGWTCH,DRADVISE,ASTHMAGE,ASATTACK,ASERVIST,ASDRVIST,ASRCHKUP,ASACTLIM,ASYMPTOM,ASNOSLEP,ASTHMED3,ASINHALR,HAREHAB1,STREHAB1,CVDASPRN,ASPUNSAF,RLIVPAIN,RDUCHART,RDUCSTRK,ARTTODAY,ARTHWGT,ARTHEXER,ARTHEDU,TETANUS,HPVADVC2,HPVADSHT,SHINGLE2,HADMAM,HOWLONG,HADPAP2,LASTPAP2,HPVTEST,HPLSTTST,HADHYST2,PROFEXAM,LENGEXAM,BLDSTOOL,LSTBLDS3,HADSIGM3,HADSGCO1,LASTSIG3,PCPSAAD2,PCPSADI1,PCPSARE1,PSATEST1,PSATIME,PCPSARS1,PCPSADE1,PCDMDECN,SCNTMNY1,SCNTMEL1,SCNTPAID,SCNTWRK1,SCNTLPAD,SCNTLWK1,SXORIENT,TRNSGNDR,RCSGENDR,RCSRLTN2,CASTHDX2,CASTHNO2,EMTSUPRT,LSATISFY,ADPLEASR,ADDOWN,ADSLEEP,ADENERGY,ADEAT1,ADFAIL,ADTHINK,ADMOVE,MISTMNT,ADANXEV,QSTVER,QSTLANG,MSCODE,_STSTR,_STRWT,_RAWRAKE,_WT2RAKE,_CHISPNC,_CRACE1,_CPRACE,_CLLCPWT,_DUALUSE,_DUALCOR,_LLCPWT,_RFHLTH,_HCVU651,_RFHYPE5,_CHOLCHK,_RFCHOL,_LTASTH1,_CASTHM1,_ASTHMS1,_DRDXAR1,_PRACE1,_MRACE1,_HISPANC,_RACE,_RACEG21,_RACEGR3,_RACE_G1,_AGEG5YR,_AGE65YR,_AGE80,_AGE_G,HTIN4,HTM4,WTKG3,_BMI5,_BMI5CAT,_RFBMI5,_CHLDCNT,_EDUCAG,_INCOMG,_SMOKER3,_RFSMOK3,DRNKANY5,DROCDY3_,_RFBING5,_DRNKWEK,_RFDRHV5,FTJUDA1_,FRUTDA1_,BEANDAY_,GRENDAY_,ORNGDAY_,VEGEDA1_,_MISFRTN,_MISVEGN,_FRTRESP,_VEGRESP,_FRUTSUM,_VEGESUM,_FRTLT1,_VEGLT1,_FRT16,_VEG23,_FRUITEX,_VEGETEX,_TOTINDA,METVL11_,METVL21_,MAXVO2_,FC60_,ACTIN11_,ACTIN21_,PADUR1_,PADUR2_,PAFREQ1_,PAFREQ2_,_MINAC11,_MINAC21,STRFREQ_,PAMISS1_,PAMIN11_,PAMIN21_,PA1MIN_,PAVIG11_,PAVIG21_,PA1VIGM_,_PACAT1,_PAINDX1,_PA150R2,_PA300R2,_PA30021,_PASTRNG,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
count,262508.000000,262508.000000,2.625080e+05,262508.000000,262508.000000,262508.000000,262508.000000,2.625080e+05,2.625080e+05,151064.0,151064.000000,25.0,151063.0,151064.000000,25.000000,151049.000000,150973.000000,150972.000000,111444.0,111444.0,111443.000000,111444.000000,629.0,111444.000000,110970.000000,108376.000000,262506.000000,262507.000000,262508.000000,134767.000000,262508.000000,262508.000000,262507.000000,262507.000000,262507.000000,105441.000000,262508.000000,227414.000000,227414.000000,262508.000000,262508.000000,35224.000000,262508.000000,262508.000000,262508.000000,262508.000000,262508.000000,262508.000000,262503.000000,33724.000000,262508.000000,262508.000000,262508.000000,262508.000000,151064.000000,7911.000000,151064.000000,262507.000000,262508.000000,262506.000000,260543.000000,259951.000000,259366.000000,259003.000000,38922.000000,256985.000000,256338.000000,255911.000000,255566.000000,255228.000000,254998.000000,254660.000000,254118.000000,109558.000000,36628.000000,72645.000000,253764.000000,253167.000000,125866.000000,125615.000000,125364.000000,245968.000000,245317.000000,244572.000000,244014.000000,243523.000000,242930.000000,241574.000000,176210.00000,175181.000000,174986.000000,174765.000000,117692.000000,11758

# Preprocess the data

In [61]:
# Preprocessing of the data (standardization, outliers, ...)
new_x_train, mean, std = preprocessing_dataset_naive(x_train)

np.set_printoptions(suppress=True, precision=3)

data = pd.DataFrame(new_x_train)

data.describe()
    

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320
count,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,262508.0,2.625080e+05,262508.0,262508.0,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,262508.0,262508.0,2.625080e+05,2.625080e+05,262508.0,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e+05,2.625080e

# Build the model

In [62]:
# Define the parameters of the algorithm.
max_iters = 50
gamma = 0.2

# Initialization
w_initial = np.zeros((new_x_train.shape[1],))

# Start gradient descent.
start_time = datetime.datetime.now()
gd_ws, gd_losses = mean_squared_error_gd(y_train, new_x_train, w_initial, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time={t:.3f} seconds".format(t=exection_time))
print("Gradient Descent: loss={l}".format(l=gd_losses))
print("Gradient Descent: w={w}".format(w=gd_ws))

GD iter. 0/49: loss=nan, w0=nan, w1=nan
GD iter. 1/49: loss=nan, w0=nan, w1=nan
GD iter. 2/49: loss=nan, w0=nan, w1=nan
GD iter. 3/49: loss=nan, w0=nan, w1=nan
GD iter. 4/49: loss=nan, w0=nan, w1=nan
GD iter. 5/49: loss=nan, w0=nan, w1=nan
GD iter. 6/49: loss=nan, w0=nan, w1=nan
GD iter. 7/49: loss=nan, w0=nan, w1=nan
GD iter. 8/49: loss=nan, w0=nan, w1=nan
GD iter. 9/49: loss=nan, w0=nan, w1=nan
GD iter. 10/49: loss=nan, w0=nan, w1=nan
GD iter. 11/49: loss=nan, w0=nan, w1=nan
GD iter. 12/49: loss=nan, w0=nan, w1=nan
GD iter. 13/49: loss=nan, w0=nan, w1=nan
GD iter. 14/49: loss=nan, w0=nan, w1=nan
GD iter. 15/49: loss=nan, w0=nan, w1=nan
GD iter. 16/49: loss=nan, w0=nan, w1=nan
GD iter. 17/49: loss=nan, w0=nan, w1=nan
GD iter. 18/49: loss=nan, w0=nan, w1=nan
GD iter. 19/49: loss=nan, w0=nan, w1=nan
GD iter. 20/49: loss=nan, w0=nan, w1=nan
GD iter. 21/49: loss=nan, w0=nan, w1=nan
GD iter. 22/49: loss=nan, w0=nan, w1=nan
GD iter. 23/49: loss=nan, w0=nan, w1=nan
GD iter. 24/49: loss=nan, 

# Test the model

In [58]:
new_x_val = preprocessing_dataset_test_naive(x_val, mean, std)
y_pred = new_x_val @ gd_ws
y_pred[y_pred <= 0] = -1
y_pred[y_pred > 0] = 1

def precision(y_pred, y_test):
    return np.sum(y_pred[y_pred == 1] == y_test[y_pred == 1]) / np.sum(y_pred == 1)

def recall(y_pred, y_test):
    return np.sum(y_pred[y_test == 1] == y_test[y_test == 1]) / np.sum(y_test == 1)

def accuracy(y_pred, y_test):
    return np.sum(y_pred == y_test) / len(y_test)

def f1_score(y_pred, y_test):
    prec = precision(y_pred, y_test)
    rec = recall(y_pred, y_test)
    return 2 * prec * rec / (prec + rec)

print("Precision: ", precision(y_pred, y_val))
print("Recall: ", recall(y_pred, y_val))
print("Accuracy: ", accuracy(y_pred, y_val))
print("F1 score: ", f1_score(y_pred, y_val))

Precision:  0.12568984946112283
Recall:  0.7681440932368003
Accuracy:  0.5189175187041919
F1 score:  0.21603098927294398


# Create Submission

In [39]:
new_x_test = preprocessing_dataset_test_naive(x_test, mean, std)
y_pred = new_x_test @ gd_ws
y_pred[y_pred <= 0] = -1
y_pred[y_pred > 0] = 1

create_csv_submission(test_ids, y_pred, "data/submission_gd.csv")